In [11]:
import numpy as np
import torch as th
import torch.autograd as ag
import torch.nn.functional as F
import torch.nn as nn
import csv
th.manual_seed(1) # set the seed 


In [12]:
fname = "/home/nafaa/Documents/Projet_TER/Sentiment Analysis Dataset.csv"
file = open(fname, "r")
labels= [] 
train = []

   
reader = csv.reader(file)

for row in reader:
       
        train.append(row[3].lstrip().strip())
        labels.append(row[1])
#supprimer la colonne sentiment et sentiment text        
train.remove('SentimentText')  
labels.remove('Sentiment')   


In [15]:
#les 50 mille dernieres données pour le test 
train_test=train[-50000:]
labels_test=labels[-50000:]
# prendre 100 mille premières données pour l'apprentissage 
train=train[:100000]
labels=labels[:100000]


In [16]:
#mettre les donnes dans un array 
train=np.asarray(train)
labels=np.asarray(labels)


In [17]:
#fonction pour séparer la phrase et avoir des mots 
def phrase_to_mot(x):
    word=""
    word_list=[]
    b=True
    for i in range(len(x)): # à l'aid de la méthode isalpha on vérifie si c'est alphabet 
        if x[i].isalpha() or x[i]=="@" or x[i]=="&":
            if b:
                word=word+x[i]
                if i+1==len(x):
                    word_list.append(word)
            else:
                b=True
                word= word+x[i]

        else:
            b=False
            if word!="":
                word_list.append(word)
            word=""
        if word=="http" or word=="www":
            word_list.append(x)
            break
   
    return word_list


['@', 'pbonjour', 'monsieur']


In [18]:
# dictionnaire contient tout les mots 
dic={}
dic_inverse={}
couts=[]
k=0
for sentense in train:
    word_list=phrase_to_mot(sentense)
    for word in word_list:
        if word not in dic:
            dic[word]=k
            k+=1
            dic_inverse[k]=word
            couts.append(1)
        else:
            index=dic[word]
            couts[index]+=1
print(len(couts))

118840


In [20]:
# un nouveau dictionnaire qui contient les mots qui apparait plus de 20 fois et moins de 10000 fois
nouveau_dic={}
nouveau_dic_inverse={}
k=1
for i in range(len(couts)):
    if (couts[i]>20):
        if (couts[i]<10000):
            word=dic_inverse[i]
            nouveau_dic[word]=k
            nouveau_dic_inverse[k]=word
            k+=1

In [21]:
vocab_size=0
for k in couts:
    if(k>20):
        if (k<10000):
            vocab_size+=1
            
print ("la taille du vocabolaire est:",vocab_size)

la taille du vocabolaire est: 4121


In [22]:
# fonction qui associé un nombre(la clé du mot dans le dictionnaire ) à chaque mot , si il n'existe pas dans le dictionnaire on le met 0
def compte(sentense):
    word_list=phrase_to_mot(sentense)
    l=[]
    for i in range(len(word_list)):
        if word_list[i] in nouveau_dic:
            k=nouveau_dic[word_list[i]]
            if k not in l:
                l.append(k)
        else:
            l.append(0)
    return l

In [23]:
print(compte(train[1]))
print(train[1])

[4, 0, 5, 6, 7, 8]
I missed the New Moon trailer...


In [24]:
#construire nouveau train et test , contient une list des tableaux convertie en entier  
nouveau_train=[]
nouveau_test=[]
for tweet in train:
    nouveau_train.append(compte(tweet))
train1=np.asarray(nouveau_train)
print(train1)

for sentense in train_test:
    nouveau_test.append(compte(sentense))
test1=np.asarray(nouveau_test)

[list([1, 2, 0, 0, 0, 3, 0]) list([4, 0, 5, 6, 7, 8]) list([9, 10, 0]) ...
 list([0, 0, 1039, 27, 341, 52, 0, 18, 0, 0, 0, 34, 164, 53, 5, 0, 0, 0, 2, 603, 0, 0, 0])
 list([0, 0, 0, 0, 0, 0, 5, 0, 75, 0])
 list([0, 193, 0, 0, 495, 0, 0, 4, 0, 0, 0, 0, 0, 0, 48, 5, 0])]


In [25]:
#convertir les labels en entier 
labels=np.asarray([int(y) for y in labels])
labels_test=np.asarray([int (y ) for y in labels_test])

In [ ]:
# binarizer 
def convert_bin (sentence): 
    v = np.zeros([vocab_size])
    for i in range(len(sentence)):
        v[sentence[i]-1]=1 
    return v



def convert_corpus_bin(corpus):
    M = np.zeros([len(corpus),vocab_size])
    i=0
    for x in corpus:
        v = convert_bin(x)
        M[i,:] = v
        i+=1
    return M

#binarizer les valeaurs du tableau
Vtrain = convert_corpus_bin(train1)

Vtest = convert_corpus_bin(test1)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
#Sequential parce que les couches de neurones vont être ajoutées séquentiellement.
model = Sequential()
#Dense parce que tous les neurones de couche précédente seront connectés à tous les neurones de la couche suivante. 
#input_dim nombre de variables prédictives
model.add(Dense(output_dim=100,input_dim=vocab_size))
model.add(Activation('relu'))
#output_dim la variable du retour
model.add(Dense(output_dim=1))
model.add(Activation('sigmoid'))

/home/nafaa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nafaa/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 12] Cannot allocate memory
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)
/home/nafaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4121, units=100)`
  after removing the cwd from sys.path.
/home/nafaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  


In [26]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
#batch_size correspond aux nombre d’observations que l’on fait passer avant de remettre à jour les poids synaptiques.
#nb_epoch est le nombre maximum d’itération
history = model.fit(Vtrain, labels,
        nb_epoch=5,
        batch_size=32,
        validation_data=(Vtest, labels_test))

/home/youssef/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 100000 samples, validate on 50000 samples
Epoch 1/5
100000/100000 [==============================] - 28s 281us/step - loss: 0.5799 - acc: 0.6947 - val_loss: 0.5796 - val_acc: 0.6943
Epoch 2/5
100000/100000 [==============================] - 28s 282us/step - loss: 0.5410 - acc: 0.7269 - val_loss: 0.5857 - val_acc: 0.6923
Epoch 3/5
100000/100000 [==============================] - 28s 281us/step - loss: 0.5099 - acc: 0.7490 - val_loss: 0.5923 - val_acc: 0.6934
Epoch 4/5
100000/100000 [==============================] - 28s 282us/step - loss: 0.4735 - acc: 0.7755 - val_loss: 0.6277 - val_acc: 0.6853
Epoch 5/5
100000/100000 [==============================] - 28s 284us/step - loss: 0.4325 - acc: 0.7994 - val_loss: 0.6414 - val_acc: 0.6898
